In [1]:
'''
challenge: resnet input: 3 channels images, the mri images: 20 channels
-> 1 layer (conv2d) to transform 16 to 3 berfore resnet model

challenge: using pretrained resent50 model and update all parameters (aksh-ai)
or just fine tune last layer
-> update all parameters
challenge: resnet-50 transform normalizing is not applicable
we have gray scale 18 channel data 
'''


'\nchallenge: resnet input: 3 channels images, the mri images: 20 channels\n-> 1 layer (conv2d) to transform 16 to 3 berfore resnet model\n\nchallenge: using pretrained resent50 model and update all parameters (aksh-ai)\nor just fine tune last layer\n-> update all parameters\nchallenge: resnet-50 transform normalizing is not applicable\nwe have gray scale 18 channel data \n'

In [2]:
import os

import numpy as np
import pandas as pd
from sklearn.utils.class_weight import compute_class_weight

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader

from config import Config, Device
from datasets import MRIDataset
from models import MriResentModel
from train import Trainer
from test_file import Tester

In [3]:
device = Device.device
print(device)

mps


In [4]:
data_path = os.path.join(os.getcwd(), "data")
labels_path = "train.csv"

batch_size = Config.batch_size
num_epochs = Config.num_epochs
learning_rate = Config.learning_rate
image_size = 256

In [5]:
train_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((image_size, image_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((image_size, image_size)),
    # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [6]:
train_dataset = MRIDataset(
    data_path, labels_path, split="train", transform=train_transforms, max_slices=20)
    
val_dataset = MRIDataset(data_path, labels_path, split="val",
                         transform=test_transforms, max_slices=20)
test_dataset = MRIDataset(
    data_path, labels_path, split="test", transform=test_transforms, max_slices=20)


train_dl = DataLoader(train_dataset, batch_size, shuffle=True)
val_dl = DataLoader(val_dataset, batch_size)
test_dl = DataLoader(test_dataset, batch_size)

In [7]:
model = MriResentModel(20,1).to(device)

/Users/fateme/Projects/iaaa-project/iaaa-venv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/fateme/Projects/iaaa-project/iaaa-venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
def compute_class_weights_from_csv(csv_file_path):
    df = pd.read_csv(csv_file_path)

    labels = df['prediction'].values

    labels = labels.astype(int)

    # Compute class weights
    unique_labels = np.unique(labels)
    class_weights = compute_class_weight(
        class_weight='balanced', classes=unique_labels, y=labels)

    # Convert to torch tensor
    return torch.tensor(class_weights, dtype=torch.float)


class_weights = compute_class_weights_from_csv(labels_path)
print(class_weights)
# For binary classification, use the appropriate class weight
# Assuming binary classification with class labels 0 and 1
class_weights = class_weights[1]  # Adjust if necessary
print("Class Weights:", class_weights)

tensor([0.5713, 4.0051])
Class Weights: tensor(4.0051)


In [9]:
class FocalLoss(nn.Module):
    """
    The focal loss for fighting against class-imbalance
    """
    def __init__(self, class_weights, device, alpha=1, gamma=2):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = 1e-12  # prevent training from Nan-loss error
        self.device = device
        
        # Ensure class_weights is a tensor and moved to the correct device
        self.class_weights = class_weights.clone().detach().to(self.device) if class_weights is not None else None

    def forward(self, logits, target):
        """
        logits & target should be tensors with shape [batch_size, num_classes]
        """
        probs = torch.sigmoid(logits)
        one_subtract_probs = 1.0 - probs
        # add epsilon
        probs_new = probs + self.epsilon
        one_subtract_probs_new = one_subtract_probs + self.epsilon
        # calculate focal loss
        log_pt = target * torch.log(probs_new) + (1.0 - target) * torch.log(one_subtract_probs_new)
        pt = torch.exp(log_pt)
        focal_loss = -1.0 * (self.alpha * (1 - pt) ** self.gamma) * log_pt


        if self.class_weights is not None:
            focal_loss = focal_loss * self.class_weights
        
        return torch.mean(focal_loss)


In [10]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)

In [11]:
model_name = model.__class__.__name__
model_name

'MriResentModel'

In [12]:
criterion = nn.BCEWithLogitsLoss().to(device)

trainer = Trainer(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    train_dl=train_dl,
    val_dl=val_dl,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    device=device,
    num_epochs=100,
    patience=50,
    threshold=0.5,
    save_path=f"saved_models/{model_name}.pth"
)

trainer.train()

100%|██████████| 59/59 [02:49<00:00,  2.87s/it]


Confusion Matrix:
[[548   0]
 [ 78   0]]
Epoch 1/100, Train Loss: 25.5248, Train Accuracy: 0.8638
Epoch 1/100, Val Accuracy: 0.8754, Precision: 0.0000, Recall: 0.0000, AUC: 0.5955, Avg Metric: 0.1985


100%|██████████| 59/59 [02:59<00:00,  3.04s/it]


Confusion Matrix:
[[548   0]
 [ 78   0]]
Epoch 2/100, Train Loss: 22.1772, Train Accuracy: 0.8739
Epoch 2/100, Val Accuracy: 0.8754, Precision: 0.0000, Recall: 0.0000, AUC: 0.6015, Avg Metric: 0.2005


100%|██████████| 59/59 [02:49<00:00,  2.88s/it]


...

Confusion Matrix:
[[514  34]
 [ 65  13]]
Epoch 53/100, Train Loss: 0.1588, Train Accuracy: 0.9995
Epoch 53/100, Val Accuracy: 0.8419, Precision: 0.2766, Recall: 0.1667, AUC: 0.6483, Avg Metric: 0.3639


100%|██████████| 59/59 [02:34<00:00,  2.62s/it]


Confusion Matrix:
[[527  21]
 [ 72   6]]
Epoch 54/100, Train Loss: 0.1793, Train Accuracy: 0.9989
Epoch 54/100, Val Accuracy: 0.8514, Precision: 0.2222, Recall: 0.0769, AUC: 0.6399, Avg Metric: 0.3130


100%|██████████| 59/59 [02:34<00:00,  2.62s/it]


Confusion Matrix:
[[514  34]
 [ 72   6]]
Epoch 55/100, Train Loss: 0.7169, Train Accuracy: 0.9963
Epoch 55/100, Val Accuracy: 0.8307, Precision: 0.1500, Recall: 0.0769, AUC: 0.6028, Avg Metric: 0.2766
Early stopping triggered


In [12]:
model = MriResentModel(20,1).to(device)
model.load_state_dict(torch.load(f"saved_models/{model_name}.pth"))

/Users/fateme/Projects/iaaa-project/iaaa-venv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/fateme/Projects/iaaa-project/iaaa-venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


<All keys matched successfully>

In [13]:
tester = Tester(
    model=model,
    test_dl=test_dl,
    test_dataset=test_dataset,
    device=device,
    threshold=0.5
)

tester.test()

Confusion Matrix:
[[307 241]
 [ 27  51]]
Test Accuracy: 0.5719, Precision: 0.1747, Recall: 0.6538, AUC: 0.6642, Avg Metric: 0.4976
